In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [4]:
df = pd.read_csv('dataset2.csv')

In [5]:
tabular_cols = ['bed', 'bath', 'sqft', 'n_citi']
target_col = 'price'

# Train/test split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Normalize tabular features
scaler = StandardScaler()
X_train_tab = scaler.fit_transform(train_df[tabular_cols])
X_val_tab = scaler.transform(val_df[tabular_cols])

# Image Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [7]:
class HousingDataset(Dataset):
    def __init__(self, df, tabular_data, transform=None):
        self.df = df.reset_index(drop=True)
        self.tabular_data = tabular_data
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.loc[idx, 'image_id']
        img_path = f"dataset1/{img_id}.jpg"
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        tabular = self.tabular_data[idx]
        target = self.df.loc[idx, 'price']

        return image, torch.tensor(tabular, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

In [8]:
train_dataset = HousingDataset(train_df, X_train_tab, transform)
val_dataset = HousingDataset(val_df, X_val_tab, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
class MultimodalModel(nn.Module):
    def __init__(self, tabular_dim):
        super(MultimodalModel, self).__init__()
        cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(cnn.children())[:-1])  # Remove FC
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128)
        )
        self.combined = nn.Sequential(
            nn.Linear(512 + 128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, images, tabular):
        img_feats = self.cnn(images)
        img_feats = img_feats.view(img_feats.size(0), -1)
        tab_feats = self.tabular(tabular)
        combined = torch.cat([img_feats, tab_feats], dim=1)
        return self.combined(combined).squeeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultimodalModel(tabular_dim=X_train_tab.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for images, tabular, targets in train_loader:
        images, tabular, targets = images.to(device), tabular.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images, tabular)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * images.size(0)

    avg_loss = epoch_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f}")




In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, tabular, targets in val_loader:
        images, tabular = images.to(device), tabular.to(device)
        outputs = model(images, tabular)
        y_true.extend(targets.numpy())
        y_pred.extend(outputs.cpu().numpy())

mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print(f"Validation MAE: {mae:.2f}, RMSE: {rmse:.2f}")
